# Классификация: CC50 > медианы

In [5]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# Подготовка целевой переменной
y = (df["CC50, mM"] > df['CC50, mM'].median()).astype(int)
X = df[features]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.2, random_state=42)
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Обучение моделей
logit = LogisticRegression(max_iter=1000)
logit.fit(X_train_scaled, y_train)
y_pred_logit = logit.predict(X_test_scaled)

rf = RandomForestClassifier()
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

xgb = XGBClassifier(use_label_encoder=False, eval_metric='logloss')
xgb.fit(X_train, y_train)
y_pred_xgb = xgb.predict(X_test)

lgb = LGBMClassifier()
lgb.fit(X_train, y_train)
y_pred_lgb = lgb.predict(X_test)

for name, preds, clf, xtest in [("Logistic Regression", y_pred_logit, logit, X_test_scaled),
                                ("Random Forest", y_pred_rf, rf, X_test),
                                ("XGBoost", y_pred_xgb, xgb, X_test),
                                ("LightGBM", y_pred_lgb, lgb, X_test)]:
    print(f"{name}: Accuracy={accuracy_score(y_test, preds):.3f}, Precision={precision_score(y_test, preds):.3f}, Recall={recall_score(y_test, preds):.3f}, F1={f1_score(y_test, preds):.3f}, AUC={roc_auc_score(y_test, clf.predict_proba(xtest)[:, 1]):.3f}")


C:\Users\USER\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [11:58:58] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[LightGBM] [Info] Number of positive: 399, number of negative: 399
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002232 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 18171
[LightGBM] [Info] Number of data points in the train set: 798, number of used features: 167
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits w

# Классификация: предсказание токсичности (CC50 > медианы)

В этом разделе мы выполняем следующие шаги:

1. **Формирование целевой переменной**  
   - Вычисляем медиану CC50 (`df["CC50, mM"].median()`).  
   - Создаём бинарный признак `y`, где 1 = “CC50 выше медианы” (мало токсично), 0 = “CC50 ≤ медианы” (более токсично).

2. **Разделение данных и масштабирование**  
   - Делаем стратифицированный сплит на обучающую и тестовую выборки (`train_test_split(..., stratify=y)`), чтобы сохранить соотношение классов.  
   - Применяем `StandardScaler` к признакам для логистической регрессии и по необходимости к другим моделям.

3. **Обучение моделей**  
   - **Логистическая регрессия** (`LogisticRegression`): базовый линейный классификатор.  
   - **Случайный лес** (`RandomForestClassifier`): модель на основе ансамбля деревьев.  
   - **XGBoost** (`XGBClassifier`): градиентный бустинг от XGBoost.  
   - **LightGBM** (`LGBMClassifier`): градиентный бустинг от LightGBM.

4. **Оценка качества**  
   Для каждой модели вычисляем на тестовом наборе:
   - Accuracy (доля правильных ответов)  
   - Precision (точность прогноза “мало токсично”)  
   - Recall (доля найденных мало токсичных)  
   - F1-score (гармоническое среднее точности и полноты)  
   - ROC AUC (площадь под ROC-кривой — оценка качества ранжирования)

5. **Сравнение и выводы**  
   - Оцениваем, какая модель даёт наилучшее сочетание метрик.  
   - Замечаем, что линейная модель часто уступает бустинговым (XGBoost/LightGBM).  
   - Анализируем важность признаков (feature importance) в деревьях: определяем, какие химические дескрипторы вносят наибольший вклад в разделение по токсичности.

> **Вывод:** модели на основе градиентного бустинга (особенно XGBoost) обычно показывают лучшие результаты в задаче разделения соединений на мало/высоко токсичные, с ROC AUC порядка ~0.80 и точностью около 0.72–0.75. Это подтверждает необходимость нелинейных методов для анализа химических данных.


In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
df = pd.read_excel("Данные_для_курсовои_Классическое_МО.xlsx")
df.drop(columns=["Unnamed: 0"], inplace=True)
constant_cols = [col for col in df.columns if df[col].nunique() == 1]
df.drop(columns=constant_cols, inplace=True)
df.dropna(inplace=True)
features = [col for col in df.columns if col not in ["IC50, mM", "CC50, mM", "SI"]]
scaler = StandardScaler()
